# Race Condition Vulnerabilities

The following PHP code introduces the example of a race condition: in banking system, when two withdraw of \$90 requests take place in a short interval, instead of accepting one and rejecting the other, the system could potentially withdraw \$180, which is undefined behavior. 

```php
function withdraw($amount)
{
    $balance = getBalance();
    if ($amount <= $balance) {
        $balance = $balance - $amount;
        echo "You have withdrawn: $amount";
        saveBalance($balance);
    }
    else {
        echo "Insufficient Fund."; 
    }
}
```